In [ ]:
#!pip install --user cmsdials[pandas,tqdm]

In [ ]:
# Standard Libraries
import os, sys, time
import pandas as pd
import numpy as np

# cmsdials utilities:
from cmsdials.auth.bearer import Credentials
from cmsdials import Dials
from cmsdials.filters import RunFilters, LumisectionFilters, MEFilters
from cmsdials.filters import LumisectionHistogram2DFilters

# Import Plotting functions:
import matplotlib.pyplot as plt
import mplhep as hep
hep.style.use("ROOT")

# Set of utility functions defined by me:
from utilities import *

## DIALS
* Autentication
* Monitoring Elements Retrieval

In [ ]:
# Autentication
creds = Credentials.from_creds_file()
# workspace definition
dials = Dials(creds, workspace="csc")

In [ ]:
# Run List: 380292, 380310, 380355, 380377, 380385, 380399, 380444

my_runs = [380292, 380310, 380355, 380377, 380385, 380399, 380444]

In [ ]:
run_list_df = dials.run.list_all(
    RunFilters(
        run_number__gte=380292,
        run_number__lte=380444
    )
).to_pandas()

In [ ]:
run_list_df.head()

In [ ]:
run_list = run_list_df["run_number"].to_numpy()

In [ ]:
my_runs_new = [i for i in my_runs if i in run_list]

In [ ]:
print(my_runs)
print(my_runs_new)

In [ ]:
from cmsdials.filters import LumisectionHistogram2DFilters
start_time = time.perf_counter()
data = []
for i, run in enumerate(my_runs_new):
    data_temp = dials.h2d.list_all(
        LumisectionHistogram2DFilters(
            me = "CSC/CSCOfflineMonitor/recHits/hRHGlobalm2",
            dataset = "/StreamExpress/Run2024D-Express-v1/DQMIO",
            run_number = run,
        ),
        enable_progress=True,
    ).to_pandas()
    data.append(data_temp)
    del data_temp
data = pd.concat(data, axis=0)
end_time = time.perf_counter()
print(f"Execution time: {end_time - start_time:.4f} seconds")

In [ ]:
out_name="data_m2.parquet"
data.to_parquet(out_name)

In [ ]:
data = pd.read_parquet(out_name)

## Pre-precessing
* Merge ME with OMS and RunRegistry info
* Selections ad siscussed in slides
* Summing of consecutive LSs

In [ ]:
# Per-LS meta-info retrieved form OMS and RunRegistry
lumi_info = pd.read_parquet("perLSinfo.parquet")

In [ ]:
#Merging MEs with OMS and RunRegistry info
monitoring_elements = pd.merge(data, lumi_info, on=['run_number', 'ls_number'], how='left') 
#Filtering (see slides)
monitoring_elements = monitoring_elements[
    (monitoring_elements["beams_stable"] == True) &
    (monitoring_elements["cscm_ready"] == True) &
    (monitoring_elements["cms_active"] == True) &
    (monitoring_elements["beam_present"] == True) &
    (monitoring_elements["physics_flag"] == True) &
    (monitoring_elements["cscSTANDBY"] == 0) &
    (monitoring_elements["cscBAD"] == 0) &
    (monitoring_elements["cscGOOD"] != 0) &
    (monitoring_elements["mean_lumi"] > 2) &
    (monitoring_elements["class"].str.contains("Collisions", na=False))
]
monitoring_elements = monitoring_elements.sort_values(by=['run_number', 'ls_number']).reset_index()
monitoring_elements = monitoring_elements.drop(columns=["index"])

In [ ]:
monitoring_elements.head()

In [ ]:
# Defining "group" column (based on Run number and instantaneous luminosity)
monitoring_elements["group"] = groupbylumi(monitoring_elements, 300)

In [ ]:
# Summing elements in the same group
summed_data = monitoring_elements.groupby("group").apply(sum_imgs)
summed_data = summed_data[summed_data["lumi"]>300]
summed_data = summed_data.reset_index()

In [ ]:
summed_data.head()

In [ ]:
Show2Dimg(summed_data["img"][0])

In [ ]:
# Removing CSC external ring
summed_data["new_img"] = summed_data.apply(
    lambda row: mask_outside_radius(row["img"], center=(49.5, 49.5), max_distance=21), axis=1
)

In [ ]:
Show2Dimg(summed_data["new_img"][0]);

## Model Predictions and Loss Computation
* Import pre-trained model
* Compute predictions and Loss mapp
* Rebinn the Loss map

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms

In [ ]:
# Loading ML model 
model = torch.jit.load("model_m2.pth")

In [ ]:
start_time = time.perf_counter()
print(f"Number of immages to be processed: {len(summed_data)}")
tensor = torch.stack([torch.tensor(m, dtype=torch.float32).unsqueeze(0) for m in summed_data["new_img"]])
loader = DataLoader(dataset=tensor, batch_size=128, num_workers=8, shuffle=False)

imgs, reco_imgs, loss_imgs = [], [], []

model.eval()
with torch.no_grad():
    for img_batch in loader:
        # Computing Model predinctions: model(img)--> reco_img 
        reco_img_batch = model(img_batch)
        # Computing Loss as (img-reco_img)/reco_img
        img_loss_batch = (img_batch - reco_img_batch)[:, 0] / reco_img_batch[:, 0]
        loss_imgs.extend(img_loss_batch.numpy())
        reco_imgs.extend(reco_img_batch[:, 0].numpy())
        imgs.extend(img_batch[:, 0].numpy())
        print(".", end="", flush=True)
end_time = time.perf_counter()
print(f"\nExecution time: {end_time - start_time:.4f} seconds")

**Curiosity:** GPUs can be used to accelerate inference. In the example above, processing 189 images took around one minute, while with GPUs, 14.5 thousand images are processed in less than 10 seconds.![GIF](https://media3.giphy.com/media/v1.Y2lkPTc5MGI3NjExMno5Y2VwcmpqMnAweGczcnkxMWRwYWdtanMxYXdjdjF0NGprYzNvaiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/26ufdipQqU2lhNA4g/giphy.gif)

In [ ]:
out_name = "processed_images_m2.npz"
np.savez(out_name, imgs=imgs, reco_imgs=reco_imgs, loss_imgs=loss_imgs)

In [ ]:
data = np.load(out_name)
imgs = data['imgs']
reco_imgs = data['reco_imgs']
loss_imgs = data['loss_imgs']

**Note:** The Loss map exhibits fluctuations that can lead to false positives. For this reason, we rebin the loss according to the geometry of the CSCs.

In [ ]:
# Example of loss
Show2DLoss(loss_imgs[1])

In [ ]:
# Rebinning Loss based on expected anomalies (see slides)
binary_matrix = (np.mean(np.sum([imgs, reco_imgs], axis=0), axis=0) != 0)
loss_imgs = [np.where(np.isinf(matrix), 2, matrix) for matrix in loss_imgs]
rebin_loss_imgs = [rebin_image(image, binary_matrix) for image in loss_imgs]

In [ ]:
# Example of rebinned-loss
Show2DLoss(rebin_loss_imgs[1])

## Study of the Maximum and Minimum of the Loss 
* Study the distribution of the maximum and minimum of the rebinned loss
* Apply thresholds
* Identify Anomalies

Note: The thresholds shown in the figures below are those obtained from the optimization described in the slides.

In [ ]:
# Computing Maximum and Minimum of the rebinned Loss
summed_data["Max"] = [np.nanmax(matrix) for matrix in rebin_loss_imgs]
summed_data["Min"] = [np.nanmin(matrix) for matrix in rebin_loss_imgs]

In [ ]:
# Distribution of the Maximum
fig, ax = plt.subplots(figsize=(8, 5))
ax.hist(summed_data["Max"], bins=70, color='red', alpha=0.7, label="Max Values")
ax.axvline(x=0.86, color='black', linestyle='dashed', linewidth=2, label="Threshold")

ax.set_xlabel("Max Loss")

plt.show()

In [ ]:
# Distribution of the Minimum
fig, ax = plt.subplots(figsize=(8, 5))
ax.hist(summed_data["Min"], bins=70, color='blue', alpha=0.7, label="Min Values")
ax.axvline(x=-0.52, color='black', linestyle='dashed', linewidth=2, label="Threshold")

ax.set_xlabel("Min Loss")

plt.show()

In [ ]:
# Application of the optimized thresholds (see slides) 
summed_data_filter = summed_data[(summed_data["Max"]>0.86) | (summed_data["Min"]<-0.52)]

In [ ]:
np.unique(summed_data_filter["run_max"])

In [ ]:
summed_data_filter

In [ ]:
show_img_reco_Loss(imgs, reco_imgs, rebin_loss_imgs, 8)

In [ ]:
#plot_LSs(monitoring_elements, 380399, (168, 182))

## Your Turn!  

Now it's your turn! Try running the code again to look for anomalies in the CSC station (-3) associated with this monitoring element:  

**CSC/CSCOfflineMonitor/recHits/hRHGlobalm3**  

Focus on the following runs: **382227, 382329, 382258, 382686** in the dataset:  

**/StreamExpress/Run2024F-Express-v1/DQMIO**  

Use the model **model_m3.pth** specifically trained for this ME